In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, explained_variance_score, mean_absolute_error

In [3]:
preds = {'rxtr': 'reactor', 'burn' : 'burnup', 'cool' : 'cooling', 'enri' : 'enrichment'}
algs = ['knn', 'dtree']
fracs = ['0.1', '0.3', '1.0']
mll_frac = '1.0'
learn15 = {}
learn29 = {}
base = '~/sims_n_results/simupdates_aug2020/'
path15 = base + 'learn15/sfcompo_compare/'
path29 = base + 'learn29/sfcompo_compare/'
path_end = '_ext_test_compare.csv'
for frac in fracs:
    for p, pred in preds.items():
        for alg in algs:
            key = p + frac + '_' + alg
            nuc_mid = 'tset' + frac + '_' + pred + '_' + alg
            pca_mid = 'pca/' + pred + '_' + alg + '_tset' + frac
            learn15['nuc_' + key] = pd.read_csv(path15 + nuc_mid + '_nuc15' + path_end, index_col=0)
            learn15['pca_' + key] = pd.read_csv(path15 + pca_mid + '_nuc15' + path_end, index_col=0)
            learn29['nuc_' + key] = pd.read_csv(path29 + nuc_mid + '_nuc29' + path_end, index_col=0)
            learn29['pca_' + key] = pd.read_csv(path29 + pca_mid + '_nuc29' + path_end, index_col=0)

nuc_mll15 = pd.read_csv(base+'sfco15/Job0_unc0.05/Job0_unc0.05.csv', index_col=0)
nuc_mll29 = pd.read_csv(base+'sfco29/Job0_unc0.05/Job0_unc0.05.csv', index_col=0)
pca_mll29 = pd.read_csv(base+'pca_sfco29/Job0_unc0.05/Job0_unc0.05.csv', index_col=0)

In [4]:
learn29['nuc_rxtr0.1_knn'].head(1)

,TrueY,kNN
0,pwr,bwr


In [5]:
Algs = ['kNN', 'DTree']

nuc_cols = ['nuc_' + col for col in Algs]
pca_cols = ['pca_' + col for col in Algs]
mll_cols = ['nuc_MaxLL', 'pca_MaxLL']

cols = nuc_cols + pca_cols + mll_cols
cols

['nuc_kNN', 'nuc_DTree', 'pca_kNN', 'pca_DTree', 'nuc_MaxLL', 'pca_MaxLL']

In [6]:
rxtr29 = pd.DataFrame(columns=cols)
for frac in fracs:
    scores29 = []
    for col in cols:
        if col in nuc_cols:
            key = 'nuc_rxtr' + frac
            if 'kNN' in col:
                key = key + '_knn'
            else:
                key = key + '_dtree'
            scores29.append(accuracy_score(learn29[key]['TrueY'], learn29[key][col[4:]]))    
        elif col in pca_cols:
            key = 'pca_rxtr' + frac
            if 'kNN' in col:
                key = key + '_knn'
            else:
                key = key + '_dtree'
            scores29.append(accuracy_score(learn29[key]['TrueY'], learn29[key][col[4:]]))
        elif col == 'nuc_MaxLL':
            if frac == mll_frac:
                scores29.append(accuracy_score(nuc_mll29['ReactorType'], nuc_mll29['pred_ReactorType']))
            else:
                scores29.append(np.nan)
        else:
            if frac == mll_frac:
                scores29.append(accuracy_score(pca_mll29['ReactorType'], pca_mll29['pred_ReactorType']))
            else:
                scores29.append(np.nan)
    rxtr29.loc[frac] = scores29
print('Reactor Type, 29 Nuclides: \n')
rxtr29

Reactor Type, 29 Nuclides: 



,nuc_kNN,nuc_DTree,pca_kNN,pca_DTree,nuc_MaxLL,pca_MaxLL
0.1,0.653465,0.352475,0.445545,0.398020,NaN,NaN
0.3,0.641584,0.801980,0.396040,0.479208,NaN,NaN
1.0,0.641584,0.627723,0.386139,0.425743,0.784158,0.554455


In [7]:
rxtr15 = pd.DataFrame(columns=nuc_cols+['nuc_MaxLL'])
for frac in fracs:
    scores15 = []
    for col in cols:
        if col in nuc_cols:
            key = 'nuc_rxtr' + frac
            if 'kNN' in col:
                key = key + '_knn'
            else:
                key = key + '_dtree'
            scores15.append(accuracy_score(learn15[key]['TrueY'], learn15[key][col[4:]]))
        elif col == 'nuc_MaxLL':
            if frac == mll_frac:
                scores15.append(accuracy_score(nuc_mll15['ReactorType'], nuc_mll15['pred_ReactorType']))
            else:
                scores15.append(np.nan)
    rxtr15.loc[frac] = scores15
print('Reactor Type, 15 Nuclides: \n')
rxtr15

Reactor Type, 15 Nuclides: 



,nuc_kNN,nuc_DTree,nuc_MaxLL
0.1,0.704950,0.635644,NaN
0.3,0.685149,0.645545,NaN
1.0,0.679208,0.647525,0.794059


In [8]:
llparam = { 'burn' : ['Burnup', 'pred_Burnup'],
            'cool' : ['CoolingTime', 'pred_CoolingTime'],
            'enri' : ['Enrichment', 'pred_Enrichment'] }

In [9]:
def explained_variances(param):
    df29 = pd.DataFrame(columns=cols)
    for frac in fracs:
        scores29 = []
        for col in cols:
            if col in nuc_cols:
                key = 'nuc_' + param + frac
                if 'kNN' in col:
                    key = key + '_knn'
                else:
                    key = key + '_dtree'
                scores29.append(explained_variance_score(learn29[key]['TrueY'], learn29[key][col[4:]]))
            elif col in pca_cols:
                key = 'pca_' + param + frac
                if 'kNN' in col:
                    key = key + '_knn'
                else:
                    key = key + '_dtree'
                scores29.append(explained_variance_score(learn29[key]['TrueY'], learn29[key][col[4:]]))    
            elif col == 'nuc_MaxLL':
                if frac == mll_frac:
                    scores29.append(explained_variance_score(nuc_mll29[llparam[param][0]], nuc_mll29[llparam[param][1]]))
                else:
                    scores29.append(np.nan)
            else:
                if frac == mll_frac:
                    scores29.append(explained_variance_score(pca_mll29[llparam[param][0]], pca_mll29[llparam[param][1]]))
                else:
                    scores29.append(np.nan)
        df29.loc[frac] = scores29
    print(llparam[param][0] + ', ExpVar, 29 nuclides: \n')
    return df29

In [10]:
def mean_abs_error(param):
    df29 = pd.DataFrame(columns=cols)
    for frac in fracs:
        scores29 = []
        for col in cols:
            if col in nuc_cols:
                key = 'nuc_' + param + frac
                if 'kNN' in col:
                    key = key + '_knn'
                else:
                    key = key + '_dtree'
                scores29.append(mean_absolute_error(learn29[key]['TrueY'], learn29[key][col[4:]]))
            elif col in pca_cols:
                key = 'pca_' + param + frac
                if 'kNN' in col:
                    key = key + '_knn'
                else:
                    key = key + '_dtree'
                scores29.append(mean_absolute_error(learn29[key]['TrueY'], learn29[key][col[4:]]))    
            elif col == 'nuc_MaxLL':
                if frac == mll_frac:
                    scores29.append(mean_absolute_error(nuc_mll29[llparam[param][0]], nuc_mll29[llparam[param][1]]))
                else:
                    scores29.append(np.nan)
            else:
                if frac == mll_frac:
                    scores29.append(mean_absolute_error(pca_mll29[llparam[param][0]], pca_mll29[llparam[param][1]]))
                else:
                    scores29.append(np.nan)
        df29.loc[frac] = scores29
    print(llparam[param][0] + ', MAE, 29 nuclides: \n')
    return df29

In [11]:
enri29 = explained_variances('enri')
enri29

Enrichment, ExpVar, 29 nuclides: 



,nuc_kNN,nuc_DTree,pca_kNN,pca_DTree,nuc_MaxLL,pca_MaxLL
0.1,-2.194820,0.761602,-1.991128,-2.111359,NaN,NaN
0.3,-2.233057,0.756433,-2.231744,-2.151357,NaN,NaN
1.0,-2.220625,0.760196,-2.250369,-2.207804,0.494476,-0.667237


In [12]:
enri29 = mean_abs_error('enri')
enri29

Enrichment, MAE, 29 nuclides: 



,nuc_kNN,nuc_DTree,pca_kNN,pca_DTree,nuc_MaxLL,pca_MaxLL
0.1,1.880246,0.273374,1.801007,1.779962,NaN,NaN
0.3,1.885808,0.271602,1.856343,1.799050,NaN,NaN
1.0,1.872577,0.268968,1.867222,1.819030,0.481465,1.324863


In [13]:
burn29 = explained_variances('burn')
burn29

Burnup, ExpVar, 29 nuclides: 



,nuc_kNN,nuc_DTree,pca_kNN,pca_DTree,nuc_MaxLL,pca_MaxLL
0.1,0.365091,-0.002214,0.121318,0.196136,NaN,NaN
0.3,0.376998,0.200148,0.100587,0.171561,NaN,NaN
1.0,0.382870,0.218227,0.068910,0.179242,0.917114,-0.188202


In [14]:
burn29 = mean_abs_error('burn')
burn29

Burnup, MAE, 29 nuclides: 



,nuc_kNN,nuc_DTree,pca_kNN,pca_DTree,nuc_MaxLL,pca_MaxLL
0.1,12719.954783,20231.556957,14744.920837,14493.669116,NaN,NaN
0.3,12659.625918,18693.649254,14853.392360,14548.225512,NaN,NaN
1.0,12592.140639,19026.505155,15000.340555,14530.088818,3611.447056,40734.010132
